In [ ]:
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat, savemat
import seaborn

# Open the data
ya = loadmat("data.mat")["y"].squeeze(0)
xa = loadmat("data.mat")["x"].squeeze(0)

# Define the number of state n
n=2

# Downsample
def downsample(t, factor):
    return (t[: len(t) // factor * factor].
            reshape((-1, factor)).mean(axis=1))

x = downsample(xa, 70)
data = downsample(ya, 70)

# Define intial and transition probabilties; and model
def init_prob(n):
    return np.full(n, 1 / n)

def trans_prob(n):
    eps = .001
    a = np.full((n, n), eps)
    for i in range(n):
        a[i, i] = 1 - eps * (n - 1)
    return a

def make_model(n):
    return GaussianHMM(n, "diag", init_prob(n), trans_prob(n))

# Run Model
model = make_model(n)
data_for_hmm = data[..., None] - data.mean()

model.fit([data_for_hmm])
print(model.transmat_)

fitted = model.predict(data_for_hmm)
savemat("Run1", {"fitted": fitted, "transitions": model.transmat_,
                       "means": model.means_, "covars": model.covars_})


# Plot  
fig, (top, mid, bot) = plt.subplots(3)
plt.xlabel('time (s)')
plt.ylabel('Bead distance variation (nm)')
top.plot(xa, ya - ya.mean())
mid.plot(x, data - data.mean())
bot.plot(x, data - data.mean())
plt.xlabel('time (s)')
plt.ylabel('Bead distance variation (nm)')
bot.plot(x, model.means_[fitted], "r", alpha=.5)
plt.show()